In [17]:
import pandas as pd
from sklearn.metrics import classification_report

In [148]:
bsq = [
    "Does the article discuss agroecological practices?",
    "Does the article cover climate change mitigation?",
    "Does the article cover climate change adaptation?",
    "Does the article comprehensively cover climate change and environmental aspects?",
    "Does the article address greenhouse gas emissions?",
    "Does the article assess agroecological practices' impact on climate change?",
    "Does the article provide relevance to stakeholders and farmers in the agricultural sector?",
    "Does the article address limitations, challenges, and potential risks?",
    "Does the article address policy implications?",
    "Does the article have a specific geographic focus?",
    "Does the article discuss the role of agroecology in enhancing climate resilience in agricultural systems?",
    "Does the article assess the impact of agroecology on nitrogen dynamics?",
    "Does the article discuss the impact of methane (CH4) emissions?"
]
len(bsq)

13

In [1]:
chatgpt_responses_b = []
for i in range(len(bsq)):
    chatgpt_responses_b.append(pd.read_excel(f"chatgpt_responses_b{i+1}.xlsx", index_col=0))
    print(i+1, bsq[i])
    print(classification_report(chatgpt_responses_b[-1]["Final decision"] == "INCLUDE", chatgpt_responses_b[-1]["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))
    print(chatgpt_responses_b[-1]["chatgpt"].apply(lambda x: int("Yes" in x)).value_counts().to_dict())

In [155]:
to_pred = chatgpt_responses_b[0].copy()
to_pred["b1"] = chatgpt_responses_b[0]["chatgpt"].apply(lambda x: int("Yes" in x))
for i in range(0, len(bsq)):
    to_pred[f"b{i+1}"] = chatgpt_responses_b[i]["chatgpt"].apply(lambda x: int("Yes" in x))

In [156]:
X = to_pred[[f"b{i+1}" for i in range(len(bsq))]]
y = (to_pred["Final decision"] == "INCLUDE").apply(int)

In [162]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

In [163]:
clf.fit(X, y)
clf.score(X, y)

0.83

In [164]:
print(classification_report(y, clf.predict(X), digits=4))

              precision    recall  f1-score   support

           0     0.8167    0.8909    0.8522        55
           1     0.8500    0.7556    0.8000        45

    accuracy                         0.8300       100
   macro avg     0.8333    0.8232    0.8261       100
weighted avg     0.8317    0.8300    0.8287       100



In [165]:
row

Publication Type                                                    J
Authors                         Rong, YP; Ma, L; Johnson, DA; Yuan, F
Article Title       Soil respiration patterns for four major land-...
Abstract (AB)       Land-use types and management practices are cr...
Publication Year                                               2015.0
DOI                                        10.1016/j.agee.2015.08.002
Researcher1                                                   INCLUDE
Researcher2                                                       NaN
Researcher3                                                   INCLUDE
Final decision                                                INCLUDE
Review studies                                                    NaN
Duplicates                                                        NaN
chatgpt                                                           No.
b1                                                                  0
b2                  

In [283]:
row = to_pred[y != clf.predict(X)].sample(1).iloc[0]
print(f"""Title: {row["Article Title"]}\nAbstract: {row["Abstract (AB)"]}\nLabel: {row["Final decision"]}""")

Title: Nitrogen-fixing trees increase soil nitrous oxide emissions: a meta-analysis
Abstract: Nitrogen-fixing trees are an important nitrogen source to terrestrial ecosystems. While they can fuel primary production and drive carbon dioxide sequestration, they can also potentially stimulate soil emissions of nitrous oxide, a potent greenhouse gas. However, studies on the influence of nitrogen-fixing trees on soil nitrous oxide emissions have not been quantitatively synthesized. Here, we show in a meta-analysis that nitrogen-fixing trees more than double soil nitrous oxide emissions relative to non-fixing trees and soils. If planted in reforestation projects at the global scale, nitrogen-fixing trees could increase global soil nitrous oxide emissions from natural terrestrial ecosystems by up to 4.1%, offsetting climate change mitigation via reforestation by up to 4.4%.
Label: INCLUDE


In [308]:
columns = "Article Title  Abstract (AB)".split("  ")
recols = {"Article Title": "t", "Abstract (AB)": "ab"}
i = 0
o = chatgpt_responses_b[i][columns].rename(columns = recols)
o["q"] = bsq[i]
o["label"] = to_pred[f"b{i+1}"]
o = o.reset_index().drop(columns="index")
for i in range(1, len(bsq)):
    if i not in [6, 4, 1]:
        oi = chatgpt_responses_b[i][columns].rename(columns = recols)
        oi["q"] = bsq[i]
        oi["label"] = to_pred[f"b{i+1}"]
        oi = oi.reset_index().drop(columns="index")
        o = pd.concat([o, oi], ignore_index=True)
o

,t,ab,q,label
0,Does carbon farming provide a cost-effective o...,"In this study, we apply a whole farm bioeconom...",Does the article discuss agroecological practi...,1
1,Influence of Organic and Mineral Fertilizers o...,The intensive use of mineral (M) fertilizers m...,Does the article discuss agroecological practi...,1
2,Climate Change Mitigation Options in the Fores...,The Intergovernmental Panel on Climate Change ...,Does the article discuss agroecological practi...,1
3,Rye cover crop incorporation and high watertab...,Drainage and cultivation of peat soils almost ...,Does the article discuss agroecological practi...,1
4,Emerging Issues and Potential Opportunities in...,The rice-wheat cropping system (RWCS) is the b...,Does the article discuss agroecological practi...,1
...,...,...,...,...
995,Consequences of field N2O emissions for the en...,One way of reducing the emissions of fossil fu...,Does the article discuss the impact of methane...,0
996,Estimation of net greenhouse gas balance using...,"The net greenhouse gas balance (NGHGB), estima...",Does the article discuss the impact of methane...,0
997,Farmers' adaptation to climate-smart agricultu...,Some of the measures to be taken to reduce gre...,Does the article discuss the impact of methane...,0
998,Valorization Methodology for Agriculture Secto...,Agriculture sector holds an essential role in ...,Does the article discuss the impact of methane...,1


In [309]:
o["label"].value_counts()

1    629
0    371
Name: label, dtype: int64

In [310]:
o.to_excel("train_bsq_abstract_chatgpt.xlsx")